# Finding Dependencies for LookML Dimensions and Measures

Before running the script, follow the instructions at https://github.com/llooker/python_api_samples to configure a config file and save it in your working directory

Import packages

In [1]:
import re
import yaml 
from lookerapi import LookerApi
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings("ignore")

Connect to your Looker Instance

In [2]:
#assumes that you the config.yml in current directory
def connect_looker():
    global looker 
    
    #get credentials
    f = open('config.yml')
    params = yaml.load(f)
    host = 'localhost'
    f.close()
    my_host = params['hosts'][host]['host']
    my_secret = params['hosts'][host]['secret']
    my_token = params['hosts'][host]['token']
    
    #connect to Looker
    looker = LookerApi(host=my_host,
    token=my_token,
    secret = my_secret)
    
    return 

Function to combine two dictionaries in a way that does not override and eliminates duplicate values 

In [3]:
def update_dictionaries(dict1,dict2,data_type):
    if data_type != 'dependencies':
        dict1.update(dict2)
    else:
        for key in dict2.keys():
            if key in dict1.keys():
                values = dict1[key]
                values = list(set(values + dict2[key]))
                dict1[key] = values
            else:
                dict1[key] = dict2[key]
    return dict1

Function to create dictionaries of Looker objects

User enters the model_name and the explore_name. If data_type is 'sql' then the resulting dictionary will have dimension/measure names as keys and sql statements as values, if data_type is 'lookml_link' then the dictionaries values will be links to the line of the metric in the view file

In [4]:
def create_dictionaries(model_name,explore_name,data_type='sql'):
    global explore
    explore = looker.get_explore(explore_name=explore_name,model_name=model_name)
    dimensions = explore['fields']['dimensions']
    measures = explore['fields']['measures']
    field_dict = dict()
    for dimension in dimensions:
        field_dict[dimension['name']] = dimension[data_type]
    for measure in measures:
        field_dict[measure['name']] = measure[data_type]
    return field_dict

For each metric in the explore environment, look at the sql values for every other metric to see if the original one is in the string

In [5]:
def compile_data(model_name,explore_name = None,data_type = 'dependencies'):
    global field_dict
    if explore_name is not None:
        data = dict()
        if data_type == 'dependencies':
            field_dict = create_dictionaries(model_name,explore_name)
            for metric in field_dict:
                for other_metric in field_dict:
                    if '${' + metric + '}' in field_dict[other_metric] or \
                        ('${' + metric.split('.')[1] + '}' in field_dict[other_metric] \
                         and metric.split('.')[0] == other_metric.split('.')[0]):
                        if metric in data.keys():
                            data[metric].append(other_metric)
                        else:
                            data[metric] = [other_metric]
                if metric not in data.keys():
                    data[metric] = [np.nan]
        elif data_type != ' dependencies':
            data = create_dictionaries(model_name,explore_name,data_type=data_type)
    else: 
        global model
        model = looker.get_model(model_name)
        data = dict()
        for explore in model['explores']:
            data = update_dictionaries(data,compile_data(model_name,explore['name'],data_type),data_type)
        
    return data

In [6]:
def transform_df(dictionary):
    df = pd.DataFrame.from_dict(dictionary,orient='index').reset_index()
    df = pd.melt(df,id_vars=['index'],value_vars=list(range(0,len(df.columns)))).drop(columns='variable')
    return df.drop_duplicates()

Create the dataframe that has the Metric, and the Dependency that is dependent on that metric, as well as the links to both

In [28]:
def create_df(model_name,explore_name=None):
    global dependencies,link_dict,link_df
    dependencies = compile_data(model_name,explore_name)
    df = transform_df(dependencies)
    
    df.columns = ['Metric','Dependency']
    
    link_dict = compile_data(model_name,explore_name,data_type='lookml_link')
    link_df = transform_df(link_dict)
    link_df.columns = ['Metric','Link']
    
    df = pd.merge(df,link_df.dropna(),on='Metric',how='left')
    df = pd.merge(df,link_df.dropna(),left_on='Dependency',right_on='Metric',how='left')
    df = df.drop(columns='Metric_y')
    df.columns = ['Metric','Dependency','Metric Link','Dependency Link']
    return df

Run the program with User inputs

In [29]:
def main():
    connect_looker()
    model_name = None
    models = looker.get_models()
    model_names = []
    for i in models:
        model_names.append(i['name'])
    while model_name not in model_names:
        print('Enter the model name:')
        model_name = input()
        if model_name not in model_names:
            print('Incorrect model name, please enter one of the following names:')
            print(' , '.join(model_names))

    model = looker.get_model(model_name)    
    explores = model['explores']
    explore_names = []
    for i in explores:
        explore_names.append(i['name'])
    explore_name = None
    while explore_name not in explore_names and explore_name != 'All':
        print('Enter the explore name, or enter "All"')
        explore_name = input()
        if explore_name not in explore_names and explore_name != 'All':
            print('Incorrect explore name, please enter one of the following names or "All":')
            print(' , '.join(explore_names))
    if explore_name == 'All':
        explore_name = None

    df = create_df(model_name,explore_name)
    df.to_csv(model_name+'_dependencies.csv')
    print('Finished! Check out the file: ' + model_name +'_dependencies.csv')
    return

In [30]:
if __name__ == "__main__":
    main()

Enter the model name:
thelook
https://demo.looker.com:19999/api/3.1/lookml_models/thelook
Enter the explore name, or enter "All"
order_items
Finished! Check out the file: thelook_dependencies.csv
